In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

,ID,url,title,label
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,0
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,0
3,3,colorbox.spb.ru,Не Беси Меня Картинки,0
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,0


In [3]:
test_df = pd.read_csv("test.csv")

test_df.head()

,ID,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [4]:
from sklearn.metrics import f1_score

In [7]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["label"].values
X_train

array(['Экс-министр экономики Молдовы - главе МИДЭИ, цель которого сделать из республики не просителя, а донора: Надо избегать долгого нахождения н�',
       'Эта песня стала известна многим телезрителям благодаря сериалу Диверсант-2',
       'Банши 4 сезон 2 серия Бремя красоты смотреть онлайн!', ...,
       '1941 Plymouth Special Deluxe Hot Rod, Automatic,Small Block,Power Steering, A/C for sale: photos, technical specifications, description',
       "Купить It's Skin Сыворотка питательная Power 10 Formula YE Effector, 30 мл в Москве - WOWCREAM",
       'Технический спирт в канистрах и флаконах, купить в Москве'],
      dtype=object)

# Simple baseline

In [9]:
y_pred = [int("порно" in str(text)) for text in X_train]

In [10]:
f1_score(y_train, y_pred)

0.3124275855120649

### Submit

In [8]:
test_df["label"] = [int("порно" in text) for text in X_test]

test_df[["ID", "label"]].to_csv("simple_baseline.csv", index=False)

!cat simple_baseline.csv | head

ID,label
135309,0
135310,0
135311,0
135312,0
135313,0
135314,0
135315,0
135316,0
135317,0
cat: stdout: Broken pipe


### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip

# ML baseline

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()

model = MultinomialNB()

In [14]:
X_train
X_train_clean = [x if isinstance(x, str) else '' for x in X_train]

In [27]:
X_train_vectorized = vectorizer.fit_transform(X_train_clean)

In [28]:
X_train_vectorized

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1202658 stored elements and shape (135309, 189852)>

In [29]:
feature_names = np.array(vectorizer.get_feature_names_out())

In [30]:
feature_names

array(['00', '000', '0000', ..., '𝔸𝕟𝕘𝕖𝕝𝕒', '𝔾𝕌𝕊𝕋𝔸𝕍', '𝕆𝕃𝔾𝔸'], dtype=object)

In [31]:
id_ = 42

print(X_train_clean[id_])

x_vector = X_train_vectorized.getrow(id_).toarray()[0]

[feature for feature in feature_names[x_vector > 0]]

Презентация на тему "Приближенное значение. Абсолютная и относительная погрешнос


['абсолютная',
 'значение',
 'на',
 'относительная',
 'погрешнос',
 'презентация',
 'приближенное',
 'тему']

In [32]:
%%time

model.fit(
    X_train_vectorized,
    y_train
)

y_pred = model.predict(
    X_train_vectorized
)

CPU times: total: 46.9 ms
Wall time: 43 ms


In [33]:
f1_score(y_train, y_pred)

0.9036641864382985

### Submit

In [18]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["label"] = model.predict(X_test_vectorized)

test_df[["ID", "label"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

ID,label
135309,0
135310,0
135311,0
135312,1
135313,0
135314,0
135315,0
135316,0
135317,0
cat: stdout: Broken pipe


In [24]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(iterations=500,
    learning_rate=0.05,
    depth=6,
    eval_metric='F1',
    early_stopping_rounds=50,
    verbose=10)
cat.fit(X_train_vectorized, y_train)

0:	learn: 0.7308057	total: 470ms	remaining: 3m 54s
10:	learn: 0.7894680	total: 4.83s	remaining: 3m 34s
20:	learn: 0.8140985	total: 9.1s	remaining: 3m 27s
30:	learn: 0.8185892	total: 13.3s	remaining: 3m 21s
40:	learn: 0.8233930	total: 17.6s	remaining: 3m 16s
50:	learn: 0.8204911	total: 21.8s	remaining: 3m 11s
60:	learn: 0.8303191	total: 26s	remaining: 3m 6s
70:	learn: 0.8377628	total: 30.2s	remaining: 3m 2s
80:	learn: 0.8426287	total: 34.4s	remaining: 2m 57s
90:	learn: 0.8415157	total: 38.6s	remaining: 2m 53s
100:	learn: 0.8517110	total: 42.8s	remaining: 2m 49s
110:	learn: 0.8543112	total: 47s	remaining: 2m 44s
120:	learn: 0.8579799	total: 51.2s	remaining: 2m 40s
130:	learn: 0.8612772	total: 55.4s	remaining: 2m 36s
140:	learn: 0.8642385	total: 59.6s	remaining: 2m 31s
150:	learn: 0.8669528	total: 1m 3s	remaining: 2m 27s
160:	learn: 0.8708917	total: 1m 8s	remaining: 2m 23s
170:	learn: 0.8734883	total: 1m 12s	remaining: 2m 18s
180:	learn: 0.8757250	total: 1m 16s	remaining: 2m 14s
190:	lear

In [35]:
y_pred2 = cat.predict(X_train_vectorized)
f1_score(y_train, y_pred2)

0.913296291510531

In [49]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["label"] = cat.predict(X_test_vectorized)

test_df[["ID", "label"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

"cat" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [52]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2')
lr.fit(X_train_vectorized, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import classification_report
y_pred3 = lr.predict(X_train_vectorized)
f1_score(y_train, y_pred3)
print(classification_report(y_train, y_pred3))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    118594
           1       1.00      0.95      0.98     16715

    accuracy                           0.99    135309
   macro avg       1.00      0.98      0.99    135309
weighted avg       0.99      0.99      0.99    135309



In [43]:
from lightgbm import LGBMClassifier

X_train_vectorized_lgbm = X_train_vectorized
X_train_vectorized_lgbm.data = X_train_vectorized_lgbm.data.astype(np.float64)

In [46]:
lgbm = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
lgbm.fit(X_train_vectorized_lgbm, y_train)

LGBMClassifier(class_weight='balanced', learning_rate=0.05, max_depth=6,
               n_estimators=200, n_jobs=-1, random_state=42)

In [47]:
y_pred4 = lgbm.predict(X_train_vectorized_lgbm)
f1_score(y_train, y_pred4)

c:\Users\Глеб\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.8944893977676144

In [51]:
X_test_vectorized_lgbm = X_test_vectorized
X_test_vectorized_lgbm.data = X_test_vectorized_lgbm.data.astype(np.float64)
test_df["label"] = lgbm.predict(X_test_vectorized)

test_df[["ID", "label"]].to_csv("ml_baseline2.csv", index=False)
!cat ml_baseline.csv | head

"cat" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
